### Predicting vessel speed in the Arctic without knowing ice conditions 

This repository documents the development of the tree-based machine-learning model for vessel speed prediction in icy waters and describes how to transform the initial training dataset (containting date-time and geographic co-ordinates) into a valid input for the model.






To begin, import the necessary libraries and load the training set(.csv file) into a dataframe. We use a small, fictional dataset for demonstration. 

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
ais_df = pd.read_csv('datasets/Cleaned_AIS_data.csv') # insert your cleaned AIS Data here, preferably in the same format
ais_df.head()

,datetime,sog,latitude,longitude
0,2017-12-09 04:37:03,0.5,71.279938,72.091477
1,2017-12-09 04:37:33,0.3,71.279948,72.091397
2,2017-12-09 04:38:32,0.0,71.279971,72.091268
3,2017-12-09 04:40:32,0.1,71.279974,72.091153
4,2017-12-09 04:48:25,1.1,71.279714,72.091150


Each row in 'ais_df' corresponds to the position of the ship at a particular point in time; 'sog' is the speed over ground (in knots).

Add extra features such as 'Landmark', 'Daylight', 'Route', and 'Season'. 
*The Landmark distance feature*, consists of distances from the ship position to four predefined geographical points.

In [3]:
from math import sin, cos, sqrt, atan2, radians

# function to compute the distance between 2 geographical co-ordinates; approximate radius of earth in km
def lat_long_dist(lati1,long1,lati2,long2):
    R = 6373.0
    lat1 = radians(lati1)
    lon1 = radians(long1)
    lat2 = radians(lati2)
    lon2 = radians(long2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [4]:
dist_port = []
for idx,row in ais_df.iterrows():
    dist_port.append(lat_long_dist(71.2733,72.0725,row.latitude,row.longitude))
ais_df['Port_Distance'] = dist_port
dist_strait = []
for idx,row in ais_df.iterrows():
    dist_strait.append(lat_long_dist(70.4807,58.2225,row.latitude,row.longitude))
ais_df['Strait_Distance'] = dist_strait
dist_oblast = []
for idx,row in ais_df.iterrows():
    dist_oblast.append(lat_long_dist(76.9550,68.4511,row.latitude,row.longitude))
ais_df['Oblast_Distance'] = dist_oblast
dist_Vilkitsky = []
for idx,row in ais_df.iterrows():
    dist_Vilkitsky.append(lat_long_dist(77.9044,102.7267,row.latitude,row.longitude))
ais_df['Vilkitsky_Distance'] = dist_Vilkitsky

In [5]:
ais_df.head()

,datetime,sog,latitude,longitude,Port_Distance,Strait_Distance,Oblast_Distance,Vilkitsky_Distance
0,2017-12-09 04:37:03,0.5,71.279938,72.091477,1.002142,511.853165,640.587551,1143.806917
1,2017-12-09 04:37:33,0.3,71.279948,72.091397,1.001055,511.850409,640.586047,1143.807716
2,2017-12-09 04:38:32,0.0,71.279971,72.091268,0.999808,511.845953,640.582911,1143.808469
3,2017-12-09 04:40:32,0.1,71.279974,72.091153,0.997336,511.841877,640.581958,1143.810627
4,2017-12-09 04:48:25,1.1,71.279714,72.091150,0.975923,511.840056,640.610564,1143.833874


*The Daylight feature* captures the effects of daylight conditions on the speed of the vessel. We use data from https://dateandtime.info/city.php?id=1538111 to get sunset and sunrise data for the studied region.

In [6]:
df = pd.read_csv('datasets/Sundata.csv')

Transform daylight data

In [7]:
from datetime import datetime

new = []
for time in df.Sunrise:
    if time != 'Polar night' and time != 'Midnight sun':
        t = time[0:5].rstrip()
        new.append(t)
    else:
        new.append(time)
df['Sunrise'] = new

new = []
for time in df.Sunset:
    if isinstance(time,str):
        t = time[0:5].rstrip()
        new.append(t)
    else:
        new.append('None')
df['Sunset'] = new

new_index = []
for date in df.Date:
    datetime_obj = datetime.strptime(date,'%a, %B %d')
    new_index.append('{0}/{1}'.format(datetime_obj.day,datetime_obj.month))
df['Date'] = new_index

new_noon = []
for time in df.SolarNoon:
    t = time[0:5].rstrip()
    new_noon.append(t)
df['SolarNoon'] = new_noon

df.set_index('Date',inplace=True, drop=True)
df.head()

,Sunrise,Sunset,SolarNoon,DayLength
Date,,,,
1/1,Polar night,None,12:15,0 h 0 m 0 s
2/1,Polar night,None,12:15,0 h 0 m 0 s
3/1,Polar night,None,12:16,0 h 0 m 0 s
4/1,Polar night,None,12:16,0 h 0 m 0 s
5/1,Polar night,None,12:17,0 h 0 m 0 s


After transforming the daylight data, take the index of the daylight data as the date/month column and introduce the Daylight feature. The Daylight feature is desiged as:


1.   Daylight conditions linearly increases from 0 to 1 from sunrise to SolarNoon
2.   Daylight conditions linearly decreases from 1 to 0 from SolarNoon to sunset
3.   Daylight remains 0 from sunset to sunrise

In [8]:
daylight = []
for date in ais_df['datetime']:
    obj = datetime.strptime(date, "%Y-%m-%d %H:%M:%S")
    idx = '{0}/{1}'.format(obj.day,obj.month)
    if df.loc[idx,'Sunrise'] == 'Polar night':
        daylight.append(0)
    elif df.loc[idx,'Sunrise'] == 'Midnight sun':
        daylight.append(1)
    else:
        sunrise_str = '{0}-{1}-{2} '.format(obj.year,obj.month,obj.day) + df.loc[idx,'Sunrise'] + ':00'
        sunrise_obj = datetime.strptime(sunrise_str, "%Y-%m-%d %H:%M:%S")
        sunnoon_str = '{0}-{1}-{2} '.format(obj.year,obj.month,obj.day) + df.loc[idx,'SolarNoon'] + ':00'
        sunnoon_obj = datetime.strptime(sunnoon_str, "%Y-%m-%d %H:%M:%S")
        sunset_str = '{0}-{1}-{2} '.format(obj.year,obj.month,obj.day) + df.loc[idx,'Sunset'] + ':00'
        sunset_obj = datetime.strptime(sunset_str, "%Y-%m-%d %H:%M:%S")
        if(obj <= sunrise_obj):
            daylight.append(0)
        elif(sunrise_obj < obj < sunnoon_obj):
            value = ((obj - sunrise_obj).total_seconds())/((sunnoon_obj - sunrise_obj).total_seconds())
            daylight.append(value)
        elif(sunnoon_obj < obj < sunset_obj):
            value = ((sunset_obj - obj).total_seconds())/((sunset_obj - sunnoon_obj).total_seconds())
            daylight.append(value)
        else:
            daylight.append(0)
ais_df['Daylight'] = daylight
ais_df.head()

,datetime,sog,latitude,longitude,Port_Distance,Strait_Distance,Oblast_Distance,Vilkitsky_Distance,Daylight
0,2017-12-09 04:37:03,0.5,71.279938,72.091477,1.002142,511.853165,640.587551,1143.806917,0
1,2017-12-09 04:37:33,0.3,71.279948,72.091397,1.001055,511.850409,640.586047,1143.807716,0
2,2017-12-09 04:38:32,0.0,71.279971,72.091268,0.999808,511.845953,640.582911,1143.808469,0
3,2017-12-09 04:40:32,0.1,71.279974,72.091153,0.997336,511.841877,640.581958,1143.810627,0
4,2017-12-09 04:48:25,1.1,71.279714,72.091150,0.975923,511.840056,640.610564,1143.833874,0


In the given example, observe that the Daylight feature is 0 for all values. This is correct since there is Polar Night December at a studied location.

*The Route feature* groups latitude, longiture into 4 sepatate non-overlaping regions.

In [9]:
route = []
for idx, row in ais_df.iterrows():
    if row['longitude'] >= 78:
        route.append(4)
    elif row['latitude'] >= 74 and row['longitude'] <= 78:
        route.append(3)
    elif row['latitude'] <= 74 and 72 <= row['longitude'] <=78:
        route.append(1)
    else:
        route.append(2)
ais_df['Route'] = route
ais_df.head()

,datetime,sog,latitude,longitude,Port_Distance,Strait_Distance,Oblast_Distance,Vilkitsky_Distance,Daylight,Route
0,2017-12-09 04:37:03,0.5,71.279938,72.091477,1.002142,511.853165,640.587551,1143.806917,0,1
1,2017-12-09 04:37:33,0.3,71.279948,72.091397,1.001055,511.850409,640.586047,1143.807716,0,1
2,2017-12-09 04:38:32,0.0,71.279971,72.091268,0.999808,511.845953,640.582911,1143.808469,0,1
3,2017-12-09 04:40:32,0.1,71.279974,72.091153,0.997336,511.841877,640.581958,1143.810627,0,1
4,2017-12-09 04:48:25,1.1,71.279714,72.091150,0.975923,511.840056,640.610564,1143.833874,0,1


*The Season feature*

In [10]:
month = []
for idx,row in ais_df.iterrows():
    time = datetime.strptime(row['datetime'],'%Y-%m-%d %H:%M:%S')
    month.append(time.month)
ais_df['month'] = month
ais_df.head()

,datetime,sog,latitude,longitude,Port_Distance,Strait_Distance,Oblast_Distance,Vilkitsky_Distance,Daylight,Route,month
0,2017-12-09 04:37:03,0.5,71.279938,72.091477,1.002142,511.853165,640.587551,1143.806917,0,1,12
1,2017-12-09 04:37:33,0.3,71.279948,72.091397,1.001055,511.850409,640.586047,1143.807716,0,1,12
2,2017-12-09 04:38:32,0.0,71.279971,72.091268,0.999808,511.845953,640.582911,1143.808469,0,1,12
3,2017-12-09 04:40:32,0.1,71.279974,72.091153,0.997336,511.841877,640.581958,1143.810627,0,1,12
4,2017-12-09 04:48:25,1.1,71.279714,72.091150,0.975923,511.840056,640.610564,1143.833874,0,1,12


In [11]:
s0 = [4,5,6]
season = []
for idx,row in ais_df.iterrows():
    if row.month in s0:
        season.append(0)
    else:
        season.append(1)
ais_df['season'] = season
ais_df.head()

,datetime,sog,latitude,longitude,Port_Distance,Strait_Distance,Oblast_Distance,Vilkitsky_Distance,Daylight,Route,month,season
0,2017-12-09 04:37:03,0.5,71.279938,72.091477,1.002142,511.853165,640.587551,1143.806917,0,1,12,1
1,2017-12-09 04:37:33,0.3,71.279948,72.091397,1.001055,511.850409,640.586047,1143.807716,0,1,12,1
2,2017-12-09 04:38:32,0.0,71.279971,72.091268,0.999808,511.845953,640.582911,1143.808469,0,1,12,1
3,2017-12-09 04:40:32,0.1,71.279974,72.091153,0.997336,511.841877,640.581958,1143.810627,0,1,12,1
4,2017-12-09 04:48:25,1.1,71.279714,72.091150,0.975923,511.840056,640.610564,1143.833874,0,1,12,1


In [12]:
ais_df.drop(['datetime','month'],axis = 1,inplace = True)
ais_df.head()

,sog,latitude,longitude,Port_Distance,Strait_Distance,Oblast_Distance,Vilkitsky_Distance,Daylight,Route,season
0,0.5,71.279938,72.091477,1.002142,511.853165,640.587551,1143.806917,0,1,1
1,0.3,71.279948,72.091397,1.001055,511.850409,640.586047,1143.807716,0,1,1
2,0.0,71.279971,72.091268,0.999808,511.845953,640.582911,1143.808469,0,1,1
3,0.1,71.279974,72.091153,0.997336,511.841877,640.581958,1143.810627,0,1,1
4,1.1,71.279714,72.091150,0.975923,511.840056,640.610564,1143.833874,0,1,1


Now, the dataset is ready to be trained on! XGBoost model was found to be one of the best among popular decision-tree based models, and this it is used for demonstration.

Divide the dataset into a training set and testing set. One can do this using the train_test_split module from scikit-learn library. We compute the accuracy of our predictions using metrics such as mean-absolute-error and r2 score.

Finally, we fit the training data into the model using the fit() and can use the model to predict speeds using the predict().

In [13]:
from xgboost import XGBRegressor 
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score,mean_absolute_error

y = ais_df.sog
x = ais_df.drop('sog', axis = 1)

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=0)

my_model = XGBRegressor(n_estimators = 100,random_state = 1,max_depth=5, gpu_id=0,eval_metric = 'mae',gamma = 0.4,min_child_weight = 5,subsample = 0.8,colsample_bytree = 0.7) 
my_model.fit(X_train, y_train) 
y_pred = my_model.predict(X_test) 
print(r2_score(y_test,y_pred),mean_absolute_error(y_test,y_pred),np.std(abs(y_test - y_pred)))

/opt/conda/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[22:36:06] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
0.9628536361209405 0.4147281477435944 0.5977403688743249
